In [9]:
import sys
sys.path.append('../src')
import json
from src.mainController import MainController
import pandas as pd
from scipy.stats import chi2_contingency

# Load the JSON data from a file
with open('../data/output_rbf_20_80.json') as f:
    json_data = json.load(f)
mainController = MainController(json_data)
gdf = mainController.dto.gdf

# Chi-Square Test of Independence between two categorical variables: 'isMeasured' (binary) and 'stability_category' (categorical with 5 levels)
# Null Hypothesis (H0): There is no relationship between the variable 'isMeasured' and the stability category '80-100%'.
# Alternative Hypothesis (H1): There is a relationship between the variable 'isMeasured' and the stability category '80-100%'.
gdf['isMeasured'] = None  
# Use boolean indexing to set values in 'isMeasured' column
gdf.loc[(gdf['all_measurements'] != 0), 'isMeasured'] = 1
gdf.loc[(gdf['all_measurements'] == 0), 'isMeasured'] = 0

#test ob is measured mit 80-100% values korreliert
# Convert 'isMeasured' to numeric (if it's not already)
gdf['isMeasured'] = pd.to_numeric(gdf['isMeasured'], errors='coerce')

# Kategorisierung der Stabilität in Prozentbereiche
gdf['stability_category'] = pd.cut(gdf['all_stability'], bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0], labels=['0-20%', '20-40%', '40-60%', '60-80%', '80-100%'])

# Numerische Kodierung der kategorialen Variable
gdf['stability_category_code'] = gdf['stability_category'].cat.codes

# Kreuztabelle erstellen
cross_table = pd.crosstab(gdf['isMeasured'], gdf['stability_category'] == '80-100%')

# Chi-Quadrat-Test durchführen
chi2, p_value, dof, expected = chi2_contingency(cross_table)

print("Chi-Square Statistic:", chi2)
print("p-value:", p_value)
# if the p-value is less 0.05, then there is evidence to reject the null hypothesis in favor of the alternative hypothesis, indicating that there is a statistically significant relationship between the variables.
# For the purposes of interpretation: a value of 15.86 can be interpreted as "moderate to strong"

Chi-Square Statistic: 15.85987915858013
p-value: 6.820951708936643e-05
